# Mock Data Pipeline

Alex Malz

[insert abstract here]

In [ ]:
import numpy as np
import bisect
import astropy.cosmology as cosmology
# from scipy.stats import norm
# import emcee
# from datetime import datetime

## Choosing true parameters

[the true redshift-dependent type rate distribution, with plot of three functions]

In [ ]:
types = {'a': 0, 'b': 1, 'c': 2}
n_types = len(types.keys())
frac_types = np.array([1. / n_types] * n_types)
assert np.isclose(np.sum(frac_types), 1.)

n_zs = 100
z_bins = np.linspace(0., 1., num=n_zs + 1, endpoint=True)
z_difs = z_bins[1:] - z_bins[:-1]

ns_of_z = []
for t in range(n_types):
    ns_of_z.append(np.ones(n_zs))

true_n_of_z = frac_types[:, np.newaxis] * np.array(ns_of_z)
assert np.isclose(np.sum(true_n_of_z * z_difs[np.newaxis, :]), 1.)

[samples of t, z from the true redshift-dependent type rate distribution]

In [ ]:
def sample_discrete(norm_dist, N):
    out_info = np.empty(shape = (N, 2)).tolist()
    assert np.isclose(np.sum(norm_dist), 1.)
    dist_shape = np.shape(norm_dist)
    flat_dist = norm_dist.flatten()
    cdf = np.cumsum(flat_dist)
    for n in range(N):
        r = np.random.random()
        k = bisect.bisect(cdf, r)
        (t_ind, z_ind) = np.unravel_index(k, shape)
        t = types[t_ind]
        z = np.random.uniform(low=z_bins[z_ind], high=z_bins[z_ind + 1])
        out_info[n][0] = t
        out_info[n][1] = z
    return out_info

n_sne = 100

discrete_n_of_z = true_n_of_z * z_difs[np.newaxis, :]
true_info = sample_discrete(discrete_n_of_z, n_sne)

[the true cosmology]

In [ ]:
H0 = 70
Om0 = 0.3

true_cosmo = cosmology.FlatLambdaCDM(H0=H0, Om0=Om0)

[true mu for each t, z]

In [ ]:
for n in range(n_sne):
    true_info[n].append(true_cosmo.distmod(true_info[n][1]))

## Making likelihoods

[the confusion matrix]

In [ ]:
conf_matrix = np.ones((n_types, n_types)) / n_types **2
assert np.isclose(np.sum(conf_matrix, axis=1).all(), frac_types.all())

[the functions taking true type, classified type, true z, and true mu returning p(z, mu | hat(t))]

[making p(z) using a simple model]

[write data to file]